In [35]:
# Basic imports

import tqdm as notebook_tqdm

import numpy as np
import importlib
import functions
import pandas as pd

# Data engineering
from sklearn.preprocessing import StandardScaler

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

# Neural Networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Functions imports
importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [36]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
x_challenge_unprocessed = pd.read_csv("../data/raw/X_test_GgyECq8.csv")
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

In [37]:
y.head(5)

,DELIVERY_START,spot_id_delta
0,2022-01-01 02:00:00+01:00,-36.874770
1,2022-01-01 03:00:00+01:00,-12.643588
2,2022-01-01 04:00:00+01:00,-1.950193
3,2022-01-01 05:00:00+01:00,1.938272
4,2022-01-01 06:00:00+01:00,0.199907


Préparation des données

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(columns=['DELIVERY_START']), y, test_size=.1)

In [39]:
x_test.head(5)

,load_forecast,coal_power_available,gas_power_available,nuclear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,hour,day_of_week,day_of_month,month,is_weekend,is_peak_hour
6889,-1.492528,-0.027836,0.995779,-0.566640,0.058365,-0.707845,-0.503496,-0.595488,-1.167643,1,1,25,10,0,0
9822,0.269885,1.143747,0.995779,0.568604,-0.359260,-0.266490,-0.753250,0.745202,0.049341,7,5,25,2,0,1
6933,-0.794831,-0.027836,0.034590,-0.686070,0.144329,-0.707845,1.724267,-0.595488,-0.153274,21,2,26,10,0,0
1217,1.405688,0.982149,0.049060,1.051706,1.066434,0.679798,-0.479123,0.167050,1.226557,9,1,22,2,0,1
1475,0.076341,1.143747,1.010248,1.022045,-0.519290,-0.707845,-0.477909,-0.595488,0.102357,3,5,5,3,0,0


In [40]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Couche de sortie pour la régression

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/env3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
# Entraînement du modèle
model.fit(x_train, y_train['spot_id_delta'], epochs=100, batch_size=32, validation_split=0.2)

# Évaluation du modèle sur l'ensemble de test
mse = model.evaluate(x_test, y_test['spot_id_delta'])
print(f'Mean Squared Error on Test Set: {mse}')

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - loss: 1819.4917 - val_loss: 974.7173
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step - loss: 2063.1479 - val_loss: 972.7306
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - loss: 1717.3191 - val_loss: 968.2651
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 1502.6162 - val_loss: 962.5804
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - loss: 1127.8595 - val_loss: 959.5595
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - loss: 1453.1357 - val_loss: 959.9280
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - loss: 1538.3000 - val_loss: 953.7562
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step - loss: 1750.9917 - val_loss: 954.3129
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - loss: 1688.3296 - val_loss: 948.4973
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - loss: 1723.2424 - val_loss: 943.3599
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - l

In [42]:
# Prédictions
y_challenge = pd.DataFrame({ # Faut prendre 'x_challenge_unprocessed' sinon il y aura le décalage horaire.
    'DELIVERY_START' : x_challenge_unprocessed['DELIVERY_START']  
    })

y_challenge['spot_id_delta'] = model.predict(x_challenge.drop(columns=['DELIVERY_START']))


y_challenge.head(5)

155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step


,DELIVERY_START,spot_id_delta
0,2023-04-02 00:00:00+02:00,5.094093
1,2023-04-02 01:00:00+02:00,2.513124
2,2023-04-02 02:00:00+02:00,0.354792
3,2023-04-02 03:00:00+02:00,0.142496
4,2023-04-02 04:00:00+02:00,-0.230233


In [43]:
y_challenge.to_csv("../data/predicted/y_challenge_11.csv", index=False)

In [ ]:
# J'ai toujours le même problème sur ce challenge. J'ai beau entraîner des modèles de plus en plus complexes,
# les résultats ne sont pas plus concluants car la corrélation entre les données d'entraînement et les données 
# de soumission n'est pas la même. Toutes les relations apprises par mes algorithmes lors de l'entraînement, sont 
# en fait inutile pour la prédiction sur les données de soumission. 

# C'est un problème d'overfitting que je n'arrive pas à régler en pré-traitement ou en prédiction. 